In [2]:
# ! pip install mlflow
# ! pip install dagshub

In [42]:
import requests
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings("ignore")

lst  = ["https://en.wikipedia.org/wiki/Predictive_text",
        "https://en.wikipedia.org/wiki/Natural_language_processing"
        # "https://en.wikipedia.org/wiki/Mahatma_Gandhi",
        # "https://en.wikipedia.org/wiki/Ratan_Tata",
        # "https://en.wikipedia.org/wiki/P._V._Sindhu",
        # "https://en.wikipedia.org/wiki/India",
        # "https://en.wikipedia.org/wiki/Deep_learning",
        # "https://en.wikipedia.org/wiki/Generative_artificial_intelligence",
        # "https://en.wikipedia.org/wiki/Amazon_(company)",
        # "https://en.wikipedia.org/wiki/Gmail",
        ]


def remove_number(text):
    return re.sub(r'\[\d+\]', '', text)

def scrap_text(lst, filename):
    all_content = []
    with open(filename, "w", encoding='utf-8') as f:
        for url in lst:
            res = requests.get(url)
            if res.status_code==200:
                soup = BeautifulSoup(res.text,'html.parser')
            
                for div in soup.find_all('div'):
                    for p in div.find_all("p"):
                        content = p.get_text().strip()
                        cleaned_content = remove_number(content)
                        # print(cleaned_content)
                        all_content.append(cleaned_content)
                        f.write(cleaned_content+"\n")
            else:
                print("Something wrong!!!")
    return all_content


scraped_text = scrap_text(lst, "text.txt")
print("Scraping complete.")


Scraping complete.


In [43]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [44]:
with open("/kaggle/working/text.txt","r") as f:
    text = f.read().strip()

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
len(tokenizer.word_index)

882

In [46]:
input_sequences = []

for sentences in text.split("\n"):
    tokenized_sent = tokenizer.texts_to_sequences([sentences])[0]
    for i in range(1, len(tokenized_sent)):
        input_sequences.append(tokenized_sent[:i+1])

In [47]:
max_len = max([len(x) for x in input_sequences])
max_len

178

In [48]:
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding="pre")

In [49]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [50]:
X.shape

(15036, 177)

In [51]:
y.shape

(15036,)

In [52]:
y = to_categorical(y, num_classes=883)
y.shape

(15036, 883)

In [53]:
# Base model
model = Sequential()

model.add(Embedding(883, 100, input_length=178))
model.add(LSTM(512, dropout=0.3))
model.add(Dense(883, activation="softmax"))

model.build(input_shape=(None,178))

In [54]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [55]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 178, 100)            │          88,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 512)                 │       1,255,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 883)                 │         452,979 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,796,703 (6.85 MB)

 Trainable params: 1,796,703 (6.85 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
import dagshub
import mlflow
import mlflow.keras

In [57]:
mlflow.set_tracking_uri("https://dagshub.com/gauravbosamiya/end-to-end-mlops-pipeline-next-word-predictor.mlflow")
dagshub.init(repo_owner="gauravbosamiya",repo_name="end-to-end-mlops-pipeline-next-word-predictor", mlflow=True)

mlflow.set_experiment("Single Layer LSTM (512) Baseline Model")

Initialized MLflow to track repo "gauravbosamiya/end-to-end-mlops-pipeline-next-word-predictor"

Repository gauravbosamiya/end-to-end-mlops-pipeline-next-word-predictor initialized!

<Experiment: artifact_location='mlflow-artifacts:/05a7f3a14fb242ba811cb728216d95e2', creation_time=1743490877624, experiment_id='0', last_update_time=1743490877624, lifecycle_stage='active', name='Single Layer LSTM (512) Baseline Model', tags={}>

In [58]:
import logging
import os 
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.info("Satrting Mlflow run...")

with mlflow.start_run():
    try:
        
        # log params
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("epochs", 5),
        # mlflow.log_param("batch_size", 32)
        mlflow.log_param("optimizer", "Adam")
        mlflow.log_param("loss_function", "categorical_crossentropy")
        
        
        logging.info("Logging model parameters...")
        mlflow.log_param("embedding_dim", 100)
        mlflow.log_param("lstm_units", 512)
        mlflow.log_param("dropout_rate", 0.3)
        mlflow.log_param("input_length", 178)
        mlflow.log_param("num_classes", 883)

        
        history = model.fit(X, y, epochs=5)
        
        
        for epoch, (train_loss, train_acc) in enumerate(zip(history.history['loss'], history.history['accuracy'])):
            mlflow.log_metric(f"train_loss_epoch_{epoch}", train_loss)
            mlflow.log_metric(f"train_accuracy_epoch_{epoch}", train_acc)
            
            
        logging.info("Saving and logging the model...")
        mlflow.keras.log_model(model,"model")
        
        
        logging.info("Model training and logging completed.")
        
    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)
         

Epoch 1/5
470/470 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.0547 - loss: 6.0885
Epoch 2/5
470/470 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.1830 - loss: 4.3213
Epoch 3/5
470/470 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.7547 - loss: 1.2335
Epoch 4/5
470/470 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9785 - loss: 0.2009
Epoch 5/5
470/470 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9892 - loss: 0.0762


2025/04/01 14:15:05 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/01 14:15:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run calm-carp-823 at: https://dagshub.com/gauravbosamiya/end-to-end-mlops-pipeline-next-word-predictor.mlflow/#/experiments/0/runs/140142bb5ae14d3c93cfdf2e55bc4948
🧪 View experiment at: https://dagshub.com/gauravbosamiya/end-to-end-mlops-pipeline-next-word-predictor.mlflow/#/experiments/0
